# Rakenna regressiomalli: valmistele ja visualisoi data

## **Lineaarinen regressio kurpitsoille - Oppitunti 2**
#### Johdanto

Nyt kun sinulla on käytössäsi työkalut koneoppimismallien rakentamiseen Tidymodelsin ja Tidyversen avulla, olet valmis aloittamaan kysymysten esittämisen datallesi. Kun työskentelet datan kanssa ja sovellat koneoppimisratkaisuja, on erittäin tärkeää ymmärtää, miten esittää oikea kysymys, jotta voit hyödyntää datasettiäsi parhaalla mahdollisella tavalla.

Tässä oppitunnissa opit:

-   Kuinka valmistella data mallin rakentamista varten.

-   Kuinka käyttää `ggplot2`:ta datan visualisointiin.

Kysymys, johon tarvitset vastauksen, määrittää, millaisia koneoppimisalgoritmeja käytät. Ja saamasi vastauksen laatu riippuu vahvasti datasi luonteesta.

Katsotaanpa tätä käytännön harjoituksen avulla.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Taiteilija: @allison_horst</figcaption>


<!--![Taiteilija: \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Taiteilija: \@allison_horst-->


## 1. Kurpitsadatan tuonti ja Tidyversen käyttöönotto

Tarvitsemme seuraavat paketit tämän oppitunnin käsittelyyn:

-   `tidyverse`: [Tidyverse](https://www.tidyverse.org/) on [kokoelma R-paketteja](https://www.tidyverse.org/packages), jotka on suunniteltu tekemään datatieteestä nopeampaa, helpompaa ja hauskempaa!

Voit asentaa ne seuraavasti:

`install.packages(c("tidyverse"))`

Alla oleva skripti tarkistaa, onko sinulla tarvittavat paketit tämän moduulin suorittamiseen, ja asentaa puuttuvat paketit puolestasi.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Nyt otetaan käyttöön joitakin paketteja ja ladataan [data](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), joka on tarjolla tätä oppituntia varten!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Pikainen `glimpse()` paljastaa heti, että datassa on tyhjiä arvoja sekä merkkijonojen (`chr`) ja numeeristen tietojen (`dbl`) sekoitus. `Date`-sarake on merkkijonotyyppiä, ja mukana on myös outo sarake nimeltä `Package`, jossa tiedot vaihtelevat `sacks`, `bins` ja muiden arvojen välillä. Data on itse asiassa melko sekavaa 😤.

Onkin melko harvinaista saada datasetti, joka olisi täysin valmis käytettäväksi ML-mallin luomiseen suoraan sellaisenaan. Mutta ei hätää, tässä oppitunnissa opit, kuinka raakadataa valmistellaan käyttäen R:n vakiokirjastoja 🧑‍🔧. Opit myös erilaisia tekniikoita datan visualisointiin. 📈📊
<br>

> Kertauksena: Pipe-operaattori (`%>%`) suorittaa operaatioita loogisessa järjestyksessä siirtämällä objektin eteenpäin funktioon tai kutsulausekkeeseen. Voit ajatella pipe-operaattorin tarkoittavan koodissasi "ja sitten".


## 2. Tarkista puuttuvat tiedot

Yksi yleisimmistä ongelmista, joita datatieteilijät kohtaavat, on puutteelliset tai puuttuvat tiedot. R edustaa puuttuvia tai tuntemattomia arvoja erityisellä tunnusarvolla: `NA` (Not Available).

Miten voisimme tietää, että data frame sisältää puuttuvia arvoja?
<br>
-   Yksi suoraviivainen tapa olisi käyttää R:n perustoimintoa `anyNA`, joka palauttaa loogiset objektit `TRUE` tai `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Hienoa, näyttää siltä, että joitakin tietoja puuttuu! Se on hyvä paikka aloittaa.

-   Toinen tapa olisi käyttää funktiota `is.na()`, joka osoittaa, mitkä yksittäiset sarake-elementit puuttuvat loogisella arvolla `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Okei, tehtävä hoidettu, mutta näin suuren tietokehyksen kohdalla olisi tehotonta ja käytännössä mahdotonta käydä läpi kaikkia rivejä ja sarakkeita yksitellen😴.

-   Intuitiivisempi tapa olisi laskea puuttuvien arvojen summa kullekin sarakkeelle:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Paljon parempi! Joitakin tietoja puuttuu, mutta ehkä sillä ei ole merkitystä tehtävän kannalta. Katsotaan, mitä lisäanalyysi tuo tullessaan.

> R:llä on erinomainen dokumentaatio mahtavien pakettien ja funktioiden lisäksi. Esimerkiksi, käytä `help(colSums)` tai `?colSums` saadaksesi lisätietoja funktiosta.


## 3. Dplyr: Tiedon käsittelyn kielioppi

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Taiteilija: @allison_horst</figcaption>


<!--![Taiteilija: \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Taiteilija: \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), Tidyversen paketti, on datan käsittelyn kielioppi, joka tarjoaa yhtenäisen joukon verbejä auttamaan yleisimpien datan käsittelyhaasteiden ratkaisemisessa. Tässä osiossa tutustumme joihinkin dplyr:n verbeihin!  
<br>


#### dplyr::select()

`select()` on `dplyr`-paketin funktio, joka auttaa valitsemaan sarakkeita säilytettäväksi tai poistettavaksi.

Jotta tietokehystäsi olisi helpompi käsitellä, poista useita sen sarakkeita käyttämällä `select()`-funktiota ja säilytä vain tarvitsemasi sarakkeet.

Esimerkiksi tässä harjoituksessa analyysimme keskittyy sarakkeisiin `Package`, `Low Price`, `High Price` ja `Date`. Valitaan nämä sarakkeet.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` on `dplyr`-paketin funktio, joka auttaa luomaan tai muokkaamaan sarakkeita säilyttäen olemassa olevat sarakkeet.

`mutate`-funktion yleinen rakenne on:

`data %>%   mutate(new_column_name = what_it_contains)`

Kokeillaan `mutate`-funktiota käyttämällä `Date`-saraketta ja tehdään seuraavat toimenpiteet:

1. Muutetaan päivämäärät (jotka ovat tällä hetkellä merkkijonoja) kuukausimuotoon (nämä ovat Yhdysvaltain päivämääriä, joten muoto on `MM/DD/YYYY`).

2. Poimitaan kuukausi päivämääristä uuteen sarakkeeseen.

R:ssä [lubridate](https://lubridate.tidyverse.org/)-paketti helpottaa päivämäärä- ja aikadatan käsittelyä. Käytetään siis `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` ja katsotaan, miten yllä olevat tavoitteet saavutetaan. Voimme poistaa Date-sarakkeen, koska emme tarvitse sitä enää myöhemmissä toimenpiteissä.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Jee! 🤩

Seuraavaksi luodaan uusi sarake `Price`, joka edustaa kurpitsan keskimääräistä hintaa. Lasketaan nyt `Low Price`- ja `High Price` -sarakkeiden keskiarvo, jotta saadaan täytettyä uusi Price-sarake.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Jep!💪

"Mutta hetkinen!", sanot selailtuasi koko aineiston `View(pumpkins)`-komennolla, "Tässä on jotain outoa!"🤔

Jos katsot `Package`-saraketta, kurpitsat myydään monissa eri muodoissa. Joitakin myydään `1 1/9 bushel` -yksiköissä, toisia `1/2 bushel` -yksiköissä, joitakin kappaleittain, joitakin painon mukaan ja joitakin suurissa laatikoissa, joiden leveydet vaihtelevat.

Varmistetaan tämä:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Upeaa!👏

Kurpitsojen punnitseminen tasaisesti näyttää olevan todella haastavaa, joten suodatetaan niitä valitsemalla vain kurpitsat, joiden `Package`-sarakkeessa on merkkijono *bushel*, ja tallennetaan tämä uuteen tietokehykseen `new_pumpkins`.


#### dplyr::filter() ja stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): luo datasta alijoukon, joka sisältää vain **rivit**, jotka täyttävät ehtosi, tässä tapauksessa kurpitsat, joiden `Package`-sarake sisältää merkkijonon *bushel*.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): tunnistaa, esiintyykö jokin kuvio merkkijonossa vai ei.

[`stringr`](https://github.com/tidyverse/stringr)-paketti tarjoaa yksinkertaisia funktioita yleisiin merkkijono-operaatioihin.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Voit nähdä, että olemme rajanneet noin 415 riviin dataa, joka sisältää kurpitsoja tynnyrillisinä.🤩  
<br>


#### dplyr::case_when()

**Mutta odota! Vielä yksi asia tehtävänä**

Huomasitko, että korimäärä vaihtelee rivikohtaisesti? Sinun täytyy normalisoida hinnoittelu niin, että näytät hinnan per kori, ei per 1 1/9 tai 1/2 kori. On aika tehdä hieman matematiikkaa standardisoinnin vuoksi.

Käytämme funktiota [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) *muuttaaksemme* Price-saraketta tiettyjen ehtojen mukaan. `case_when` mahdollistaa useiden `if_else()`-lauseiden vektorisoinnin.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nyt voimme analysoida yksikköhinnan perustuen niiden mittaukseen bushel-mitoilla. Kaikki tämä kurpitsabushelien tutkiminen kuitenkin osoittaa, kuinka `tärkeää` on `ymmärtää datasi luonne`!

> ✅ [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) -sivuston mukaan bushelin paino riippuu tuotteen tyypistä, koska se on tilavuusmitta. "Esimerkiksi tomaattibushelin pitäisi painaa 56 paunaa... Lehdet ja vihreät vievät enemmän tilaa mutta painavat vähemmän, joten pinaattibushel painaa vain 20 paunaa." Tämä on aika monimutkaista! Ei vaivauduta tekemään bushelista paunoihin -muunnosta, vaan hinnoitellaan bushelin mukaan. Kaikki tämä kurpitsabushelien tutkiminen kuitenkin osoittaa, kuinka tärkeää on ymmärtää datasi luonne!
>
> ✅ Huomasitko, että puolibushelilla myydyt kurpitsat ovat todella kalliita? Voitko selvittää miksi? Vinkki: pienet kurpitsat ovat paljon kalliimpia kuin isot, luultavasti siksi, että niitä on bushelissa paljon enemmän, kun yksi iso ontto piirakkakurpitsa vie käyttämätöntä tilaa.


Nyt viimeiseksi, seikkailun vuoksi 💁‍♀️, siirretään Kuukausi-sarake ensimmäiseksi eli `ennen` saraketta `Paketti`.

`dplyr::relocate()` -funktiota käytetään sarakkeiden sijainnin muuttamiseen.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Hyvää työtä!👌 Sinulla on nyt siisti ja järjestelmällinen aineisto, jonka pohjalta voit rakentaa uuden regressiomallisi!
<br>


## 4. Datavisualisointi ggplot2:lla

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografiikka: Dasani Madipalli</figcaption>


<!--![Infografiikka: Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

On olemassa *viisas* sanonta, joka kuuluu näin:

> "Yksinkertainen kaavio on tuonut enemmän tietoa data-analyytikon mieleen kuin mikään muu väline." --- John Tukey

Osana data-analyytikon työtä on osoittaa datan laatu ja luonne, jonka parissa he työskentelevät. Tätä varten he usein luovat kiinnostavia visualisointeja, kuten kaavioita, graafeja ja diagrammeja, jotka esittävät datan eri puolia. Näin he voivat visuaalisesti näyttää suhteita ja aukkoja, jotka muuten olisivat vaikeasti havaittavissa.

Visualisoinnit voivat myös auttaa määrittämään, mikä koneoppimistekniikka sopii parhaiten datalle. Esimerkiksi hajontakaavio, joka näyttää seuraavan suoraa linjaa, viittaa siihen, että data sopii hyvin lineaarisen regressioharjoituksen kohteeksi.

R tarjoaa useita järjestelmiä graafien luomiseen, mutta [`ggplot2`](https://ggplot2.tidyverse.org/index.html) on yksi tyylikkäimmistä ja monipuolisimmista. `ggplot2` mahdollistaa graafien koostamisen **yhdistämällä itsenäisiä komponentteja**.

Aloitetaan yksinkertaisella hajontakaaviolla, joka kuvaa Price- ja Month-sarakkeita.

Tässä tapauksessa aloitamme [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html)-funktiolla, annamme sille datasetin ja esteettisen kartoituksen (käyttäen [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) ja lisäämme kerroksia (kuten [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) hajontakaavioita varten.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Onko tämä hyödyllinen kaavio 🤷? Yllättääkö siinä jokin sinua?

Se ei ole erityisen hyödyllinen, sillä se vain näyttää tietosi pistejoukkona tietyn kuukauden aikana.
<br>


### **Kuinka teemme siitä hyödyllisen?**

Jotta kaaviot näyttäisivät hyödyllistä dataa, sinun täytyy yleensä ryhmitellä data jollain tavalla. Esimerkiksi meidän tapauksessamme, keskimääräisen kurpitsan hinnan löytäminen kuukausittain antaisi enemmän tietoa datamme taustalla olevista kuvioista. Tämä johdattaa meidät yhteen **dplyr**-pikakatsaukseen:

#### `dplyr::group_by() %>% summarize()`

R:ssä ryhmitelty aggregointi voidaan helposti laskea käyttämällä

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` muuttaa analyysin yksikön koko datasetistä yksittäisiin ryhmiin, kuten kuukausittain.

-   `dplyr::summarize()` luo uuden dataframen, jossa on yksi sarake kutakin ryhmittelymuuttujaa varten ja yksi sarake kutakin määrittelemääsi yhteenvetotilastoa varten.

Esimerkiksi voimme käyttää `dplyr::group_by() %>% summarize()` ryhmitelläksemme kurpitsat **Kuukausi**-sarakkeen perusteella ja sitten laskea **keskimääräisen hinnan** kullekin kuukaudelle.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Ytimekkäästi!✨

Kategoriset ominaisuudet, kuten kuukaudet, esitetään paremmin pylväsdiagrammin avulla 📊. Pylväsdiagrammeista vastaavat kerrokset ovat `geom_bar()` ja `geom_col()`. Katso `?geom_bar` saadaksesi lisätietoja.

Tehdään yksi!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Tämä on hyödyllisempi datavisualisointi! Näyttää siltä, että kurpitsojen korkein hinta esiintyy syys- ja lokakuussa. Vastaako tämä odotuksiasi? Miksi tai miksi ei?

Onnittelut toisen oppitunnin suorittamisesta 👏! Valmistelit datasi mallin rakentamista varten ja löysit lisää oivalluksia visualisointien avulla!



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskääntämistä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
